In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [11]:
df = pd.read_excel('/Users/quinnmackay/Documents/GitHub/BICC/Data Storage/Tiepoints/Svensson Links NGRIP-NEEM-GIP-GISP-EDML-EDC-WD-DJ-TALDICE.xls', sheet_name=1)
df_trim = df[['NGRIP_depth(m)', 'WD_depth(m)']]
df_trim = df_trim.dropna()
df_trim.reset_index(drop=True, inplace=True)
df_trim = df_trim[:192]
df_trim

df_trim = df_trim.rename(columns={'NGRIP_depth(m)': 'depth1', 'WD_depth(m)': 'depth2'})

df_trim['age_unc'] = 40

df_trim

df_trim2 = df_trim[['depth1', 'depth2', 'age_unc']]

df_trim2.to_csv('/Users/quinnmackay/Documents/GitHub/BICC/Paleochrono BICC Work/Paleochrono BICC Experiment/BICC2025/NGRIP-WDC/iceice_synchro_horizons.txt', index=False, sep='\t')